# Предикт на сабмит


In [4]:
# установка последнего lightautoml
# !pip install -q git+https://github.com/sb-ai-lab/LightAutoML.git

In [5]:
import os
if 'drive' not in os.listdir():
    from google.colab import drive
    drive.mount('/content/drive')

path_to_folder = '/content/drive/MyDrive/psb_hack'

Mounted at /content/drive


In [6]:
# standard libraries
import sys
import os
import json
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
# ds libraries
import pandas as pd
import numpy as np
# lama
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
# save/load model
import joblib
import gc
import yaml

### Загрузка конфига и данных

In [7]:
print("Доступные конфиги модели:\n")
sorted(filter(lambda x: 'lama' in x, os.listdir(os.path.join(path_to_folder, 'models'))))

Доступные конфиги модели:



['psb_hack_lama_v511.pkl',
 'psb_hack_lama_v521.pkl',
 'psb_hack_lama_v531.pkl',
 'psb_hack_lama_v532.pkl',
 'psb_hack_lama_v533.pkl',
 'psb_hack_lama_v541.pkl',
 'psb_hack_lama_v542.pkl',
 'psb_hack_lama_v543.pkl']

In [8]:
# конфиг модели
config_name = 'psb_hack_lama_v541.yaml'

In [9]:
# конфиг модели
with open(os.path.join(path_to_folder, 'configs', config_name),'r',encoding='utf-8') as config:
    config_model = yaml.full_load(config)
# нужные файлы
full_train_flag = config_model['full_train_flag']
features_pack = config_model['features_pack']
model_file =  os.path.join(path_to_folder,'models',config_model['model_name'] + '.pkl')
metric_file =  os.path.join(path_to_folder,'metrics',config_model['model_name'] + '_metrics.xlsx')

In [12]:
# чтение данных
dates = ['Дата бронирования','Заезд', 'Выезд']
read_cols = None
test = pd.read_csv(os.path.join(path_to_folder,'data',f'test_{features_pack}.csv'),parse_dates=dates,usecols=read_cols)
gc.collect()
# загрузим модель

with open(model_file,'rb') as save_model:
    model = joblib.load(save_model)
print(model.create_model_str_desc())

Final prediction for new objects (level 0) = 
	 0.29323 * (5 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.08642 * (5 averaged models Lvl_0_Pipe_1_Mod_0_LightGBM) +
	 0.33736 * (5 averaged models Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM) +
	 0.11513 * (5 averaged models Lvl_0_Pipe_1_Mod_2_CatBoost) +
	 0.11144 * (5 averaged models Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost) +
	 0.05642 * (5 averaged models Lvl_0_Pipe_1_Mod_5_Tuned_XGBoost) 


In [13]:
# загрузка теста и шаблона сэмпла
sample = pd.read_csv(os.path.join(path_to_folder,'data','example.csv'),names = ['pred'])
# предикты и сохранение
sample['pred'] = np.around(model.predict(test).data.reshape(-1),3)#(model.predict(test).data.reshape(-1) >= thr).astype(int)
sample.to_csv(os.path.join(path_to_folder,'submits',config_model['model_name'] + '_sample.csv'),header = False,index=False)
# проверка что все сохранилось
pd.read_csv(os.path.join(path_to_folder,'submits',config_model['model_name'] + '_sample.csv'),names=['pred']).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11218 entries, 0 to 11217
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   pred    11218 non-null  float64
dtypes: float64(1)
memory usage: 87.8 KB
